In [1]:
push!(LOAD_PATH, "/home/behrouz/joey/Docs/Personal/uncertainty-reasoning/my_engine/julia_engine")
using ParseNet

In [2]:
node_list, potential_list = parse_net("data/asia.net")

(ParseNet.Node[ParseNet.Node("asia",String["yes","no"]),ParseNet.Node("tub",String["yes","no"]),ParseNet.Node("smoke",String["yes","no"]),ParseNet.Node("lung",String["yes","no"]),ParseNet.Node("bronc",String["yes","no"]),ParseNet.Node("either",String["yes","no"]),ParseNet.Node("xray",String["yes","no"]),ParseNet.Node("dysp",String["yes","no"])],ParseNet.Potential[ParseNet.Potential("asia",String[],[0.01,0.99]),ParseNet.Potential("tub",String["asia"],[0.05,0.95,0.01,0.99]),ParseNet.Potential("smoke",String[],[0.5,0.5]),ParseNet.Potential("lung",String["smoke"],[0.1,0.9,0.01,0.99]),ParseNet.Potential("bronc",String["smoke"],[0.6,0.4,0.3,0.7]),ParseNet.Potential("either",String["lung","tub"],[1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0]),ParseNet.Potential("xray",String["either"],[0.98,0.02,0.05,0.95]),ParseNet.Potential("dysp",String["bronc","either"],[0.9,0.1,0.8,0.2,0.7,0.3,0.1,0.9])])

In [3]:
# Returns a moral graph as an adjacency matrix
function create_moral_graph(node_list::Array{Node,1}, 
                potential_list::Array{Potential,1})
    a_list = Dict{String, Set{String}}()
    # initialize with empty set
    for n in node_list
        a_list[n.name] = Set{String}()
    end
    
    function add_edge(i, j)
        push!(a_list[i], j)
        push!(a_list[j], i)
    end
    
    for p in potential_list
        for o in p.other_nodes
            add_edge(p.node, o)
        end
        for i in 1:length(p.other_nodes)
            for j in (i+1):length(p.other_nodes)
                add_edge(p.other_nodes[i], p.other_nodes[j])
            end
        end
    end
    return a_list
end

create_moral_graph (generic function with 1 method)

In [4]:
create_moral_graph(node_list, potential_list)

Dict{String,Set{String}} with 8 entries:
  "bronc"  => Set(String["either","dysp","smoke"])
  "asia"   => Set(String["tub"])
  "lung"   => Set(String["either","tub","smoke"])
  "either" => Set(String["bronc","lung","dysp","tub","xray"])
  "dysp"   => Set(String["bronc","either"])
  "tub"    => Set(String["either","asia","lung"])
  "xray"   => Set(String["either"])
  "smoke"  => Set(String["bronc","lung"])

In [18]:
function get_next_node(g::Dict{String,Set{String}}, 
    node_weights::Dict{String,Int})
    # for all nodes, compute the number of edges to be added 
    # and the cluster weight
    best_f = 0
    best_w = 0
    best_k = 0
    function update(f, w, k)
        best_f = f
        best_w = w
        best_k = k
    end

    nodes =  collect(keys(g))
    first = true
    for (k, node) in enumerate(nodes)
        nei = collect(g[node])
        f = 0
        w = node_weights[node]
        for i in 1:length(nei)
            w += node_weights[nei[i]]
            for j in (i+1):length(nei)
                if !(nei[i] in g[nei[j]])
                    f += 1
                end
            end
        end
        if first
            first = false
            update(f, w, k)
        else
            if (f < best_f) || (f == best_f && w < best_w)
                update(f, w, k)
            end
        end
    end
    return nodes[best_k]
end

get_next_node (generic function with 1 method)

In [21]:
# TODO use a binary heap for getting the next node
# footnote 7 in page 12 of Huang and Darwiche
function triangulate_graph(g::Dict{String,Set{String}}, 
    node_list::Array{Node, 1})
    g1 = deepcopy(g)
    g2 = deepcopy(g)    
    function add_edge(i, j)
        for g in (g1, g2)
            push!(g[i], j)
            push!(g[j], i)
        end
    end
    
    node_weights = Dict{String, Int}()
    for node in node_list
        node_weights[node.name] = length(node.states)
    end
    
    clusters = Array{Set{String}, 1}()
    function check_add(g::Dict{String, Set{String}}, node::String)
        cluster = g[node]
        push!(cluster, node)
        subsumed = false
        for c in clusters
            if issubset(cluster, c)
                subsumed = true
                break
            end
        end
        if !subsumed
            push!(clusters, cluster)
        end
    end
    
    #TODO add clusters
    while !isempty(keys(g1))
        next_node = get_next_node(g1, node_weights)
        println(next_node)
        nei = collect(g1[next_node])
        for i in 1:length(nei)
            for j in (i+1):length(nei)
                add_edge(nei[i], nei[j])
            end
            delete!(g1[nei[i]], next_node)
        end
        delete!(g1, next_node)
    end
    return g2, clusters
end

triangulate_graph (generic function with 1 method)

In [22]:
x = create_moral_graph(node_list, potential_list)

Dict{String,Set{String}} with 8 entries:
  "bronc"  => Set(String["either","dysp","smoke"])
  "asia"   => Set(String["tub"])
  "lung"   => Set(String["either","tub","smoke"])
  "either" => Set(String["bronc","lung","dysp","tub","xray"])
  "dysp"   => Set(String["bronc","either"])
  "tub"    => Set(String["either","asia","lung"])
  "xray"   => Set(String["either"])
  "smoke"  => Set(String["bronc","lung"])

In [23]:
y = triangulate_graph(x, node_list)

asia
xray
dysp
tub
bronc
lung
either
smoke


Dict{String,Set{String}} with 8 entries:
  "bronc"  => Set(String["either","dysp","smoke"])
  "asia"   => Set(String["tub"])
  "lung"   => Set(String["either","tub","smoke"])
  "either" => Set(String["bronc","lung","dysp","tub","xray","smoke"])
  "dysp"   => Set(String["bronc","either"])
  "tub"    => Set(String["either","asia","lung"])
  "xray"   => Set(String["either"])
  "smoke"  => Set(String["bronc","either","lung"])

In [24]:
x = Array{Set{String}, 1}()

0-element Array{Set{String},1}

In [25]:
push!(x, Set(["This", "That"]))

1-element Array{Set{String},1}:
 Set(String["That","This"])

In [1]:
t = Set([1, 2, 3])

Set([2,3,1])

In [3]:
push!(t, 4)

Set([4,2,3,1])

In [4]:
issubset(t, Set([1, 2, 3, 4, 5]))

true